# Monarch Watch Data all taggings GBIF

In [1]:
#  Import and read
import pandas as pd 
import numpy as np
import json
# Sue Mottet 2-11-2022

In [2]:
MWatch_Tagged_df = pd.read_csv("MWatch_occurrences_1996_2000.csv")
MWatch_Tagged_df.head()

,id,modified,language,license,institutionID,collectionID,institutionCode,collectionCode,datasetName,basisOfRecord,...,decimalLatitude,decimalLongitude,geodeticDatum,georeferenceRemarks,taxonID,nameAccordingToID,scientificName,kingdom,vernacularName,taxonRemarks
0,MW0000001,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus
1,MW0000002,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus
2,MW0000003,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus
3,MW0000004,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus
4,MW0000005,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus


In [3]:
MWatch_Tagged_df.columns.tolist()

['id',
 'modified',
 'language',
 'license',
 'institutionID',
 'collectionID',
 'institutionCode',
 'collectionCode',
 'datasetName',
 'basisOfRecord',
 'occurrenceID',
 'catalogNumber',
 'recordedBy',
 'occurrenceStatus',
 'associatedReferences',
 'eventDate',
 'year',
 'country',
 'stateProvince',
 'verbatimLocality',
 'decimalLatitude',
 'decimalLongitude',
 'geodeticDatum',
 'georeferenceRemarks',
 'taxonID',
 'nameAccordingToID',
 'scientificName',
 'kingdom',
 'vernacularName',
 'taxonRemarks']

In [4]:
def df_to_geojson(df, properties, lat='decimalLatitude', lon='decimalLongitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [5]:
MWatch_Tagged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363582 entries, 0 to 363581
Data columns (total 30 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    363582 non-null  object 
 1   modified              363582 non-null  object 
 2   language              363582 non-null  object 
 3   license               363582 non-null  object 
 4   institutionID         363582 non-null  object 
 5   collectionID          363582 non-null  object 
 6   institutionCode       363582 non-null  object 
 7   collectionCode        363582 non-null  object 
 8   datasetName           363582 non-null  object 
 9   basisOfRecord         363582 non-null  object 
 10  occurrenceID          363582 non-null  object 
 11  catalogNumber         363582 non-null  object 
 12  recordedBy            363582 non-null  object 
 13  occurrenceStatus      363582 non-null  object 
 14  associatedReferences  363582 non-null  object 
 15  

In [6]:
MWatch_Tagged_df.describe()

,year,decimalLatitude,decimalLongitude,taxonID
count,363582.000000,363582.000000,363582.000000,363582.0
mean,1997.858071,39.694501,-90.112869,117273.0
std,1.763730,3.769477,8.696296,0.0
min,1992.000000,25.721490,-118.997683,117273.0
25%,1997.000000,38.203070,-96.731103,117273.0
50%,1998.000000,39.216660,-93.664964,117273.0
75%,1999.000000,42.213930,-82.491268,117273.0
max,2000.000000,49.895140,-67.958850,117273.0


In [7]:
print(MWatch_Tagged_df.shape)

(363582, 30)


In [8]:
# Determine the number of unique values in each column.
for col in MWatch_Tagged_df:
  print(MWatch_Tagged_df[col].unique())

['MW0000001' 'MW0000002' 'MW0000003' ... 'MW0363580' 'MW0363581'
 'MW0363582']
['7/30/2019']
['English']
['https://creativecommons.org/publicdomain/zero/1.0/legalcode']
['https://bison.usgs.gov/']
['https://bison.usgs.gov/ipt/resource?r=monarchwatch']
['BISON']
['Monarch Watch']
['Monarch Watch']
['humanobservation']
['MW0000001' 'MW0000002' 'MW0000003' ... 'MW0363580' 'MW0363581'
 'MW0363582']
['MW0000001' 'MW0000002' 'MW0000003' ... 'MW0363580' 'MW0363581'
 'MW0363582']
['Monarch Watch']
['present']
['Monarch Watch Tagging Data, https://monarchwatch.org/tagdata']
['8/9/1992' '8/10/1992' '9/1/1992' ... '12/29/2000' '12/30/2000'
 '12/31/2000']
[1992 1993 1994 1995 1996 1997 1998 1999 2000]
['US' 'CA']
['MD' 'MO' 'KS' 'MI' 'KY' 'CT' 'NJ' 'CO' 'TX' 'OH' 'NY' 'MA' 'GA' 'IN'
 'PA' 'NC' 'MN' 'ON' 'NE' 'OK' 'FL' 'NH' 'VT' 'IA' 'IL' 'QC' 'WI' 'AZ'
 'VA' 'ME' 'WV' 'TN' 'SD' 'LA' 'RI' 'DE' 'AL' 'NM' 'AR' 'SC' 'ND' 'DC'
 'MS' 'MB' 'WY' 'BC' 'UT' 'ID']
['City:PLUM POINT' 'City:INDEPENDENCE' 'City

In [9]:
# Determine the number of unique values in each column.
for col in MWatch_Tagged_df:
  print(MWatch_Tagged_df[col].value_counts())

MW0000001    1
MW0242418    1
MW0242394    1
MW0242393    1
MW0242392    1
            ..
MW0121193    1
MW0121192    1
MW0121191    1
MW0121190    1
MW0363582    1
Name: id, Length: 363582, dtype: int64
7/30/2019    363582
Name: modified, dtype: int64
English    363582
Name: language, dtype: int64
https://creativecommons.org/publicdomain/zero/1.0/legalcode    363582
Name: license, dtype: int64
https://bison.usgs.gov/    363582
Name: institutionID, dtype: int64
https://bison.usgs.gov/ipt/resource?r=monarchwatch    363582
Name: collectionID, dtype: int64
BISON    363582
Name: institutionCode, dtype: int64
Monarch Watch    363582
Name: collectionCode, dtype: int64
Monarch Watch    363582
Name: datasetName, dtype: int64
humanobservation    363582
Name: basisOfRecord, dtype: int64
MW0000001    1
MW0242418    1
MW0242394    1
MW0242393    1
MW0242392    1
            ..
MW0121193    1
MW0121192    1
MW0121191    1
MW0121190    1
MW0363582    1
Name: occurrenceID, Length: 363582, dtype: int6

In [10]:
# Find null values
for column in MWatch_Tagged_df.columns:
   print(f"Column {column} has {MWatch_Tagged_df[column].isnull().sum()} null values")

Column id has 0 null values
Column modified has 0 null values
Column language has 0 null values
Column license has 0 null values
Column institutionID has 0 null values
Column collectionID has 0 null values
Column institutionCode has 0 null values
Column collectionCode has 0 null values
Column datasetName has 0 null values
Column basisOfRecord has 0 null values
Column occurrenceID has 0 null values
Column catalogNumber has 0 null values
Column recordedBy has 0 null values
Column occurrenceStatus has 0 null values
Column associatedReferences has 0 null values
Column eventDate has 0 null values
Column year has 0 null values
Column country has 0 null values
Column stateProvince has 0 null values
Column verbatimLocality has 0 null values
Column decimalLatitude has 0 null values
Column decimalLongitude has 0 null values
Column geodeticDatum has 0 null values
Column georeferenceRemarks has 0 null values
Column taxonID has 0 null values
Column nameAccordingToID has 0 null values
Column scienti

In [11]:
# Find duplicate entries
print(f"Duplicate entries: {MWatch_Tagged_df.duplicated().sum()}")

Duplicate entries: 0


# Format file for geoJSON

In [12]:
MWatch_Tagged_df['species'] = 'Monarch Tagged'

In [13]:
MWatch_Tagged_df.head()

,id,modified,language,license,institutionID,collectionID,institutionCode,collectionCode,datasetName,basisOfRecord,...,decimalLongitude,geodeticDatum,georeferenceRemarks,taxonID,nameAccordingToID,scientificName,kingdom,vernacularName,taxonRemarks,species
0,MW0000001,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged
1,MW0000002,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged
2,MW0000003,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged
3,MW0000004,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged
4,MW0000005,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged


In [14]:
MWatch_Tagged_df.drop(['modified', 'language', 'license', 'institutionID', 'collectionID', 'institutionCode', 'collectionCode', 'datasetName', 'basisOfRecord', 'occurrenceID', 'catalogNumber', 'recordedBy', 'occurrenceStatus', 'associatedReferences', 'verbatimLocality', 'geodeticDatum', 'georeferenceRemarks', 'taxonID', 'nameAccordingToID','scientificName','kingdom', 'vernacularName','taxonRemarks'], axis=1, inplace=True)
MWatch_Tagged_df.head()

,id,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species
0,MW0000001,8/9/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
1,MW0000002,8/9/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
2,MW0000003,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
3,MW0000004,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
4,MW0000005,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged


In [87]:
cols = ['eventDate', 'species','country', 'stateProvince', 'decimalLatitude', 'decimalLongitude', 'number']

In [16]:
MWatch_Tagged_df.shape

(363582, 8)

In [17]:
# MWatch_Tagged_df.replace(np.nan,'',regex=True)

In [18]:
MWatch_Tagged_df.shape

(363582, 8)

In [19]:
MWatch_Tagged_df

,id,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species
0,MW0000001,8/9/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
1,MW0000002,8/9/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
2,MW0000003,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
3,MW0000004,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
4,MW0000005,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
...,...,...,...,...,...,...,...,...
363577,MW0363578,12/30/2000,2000,US,TX,29.50745,-95.094930,Monarch Tagged
363578,MW0363579,12/31/2000,2000,US,TX,29.50745,-95.094930,Monarch Tagged
363579,MW0363580,12/31/2000,2000,US,TX,29.50745,-95.094930,Monarch Tagged
363580,MW0363581,12/31/2000,2000,US,TX,29.50745,-95.094930,Monarch Tagged


In [20]:
# MWatch_Tagged_df['column name'] = df['column name'].replace(['old value'],'new value')

In [21]:
MWatch_Tagged_df =MWatch_Tagged_df.groupby(['eventDate', 'year', 'country', 'stateProvince', 'decimalLatitude', 'decimalLongitude', 'species']).size().reset_index()
MWatch_Tagged_df.head()

,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species,0
0,1/1/2000,2000,US,FL,28.01863,-82.112864,Monarch Tagged,1
1,1/1/2000,2000,US,FL,28.53834,-81.379237,Monarch Tagged,1
2,1/10/1998,1998,US,KS,37.58613,-99.475674,Monarch Tagged,9
3,1/10/1998,1998,US,TX,25.90175,-97.497484,Monarch Tagged,1
4,1/10/1998,1998,US,TX,29.76043,-95.369803,Monarch Tagged,7


In [22]:
MWatch_Tagged_df.rename( columns={0:'number'} ,inplace=True)
MWatch_Tagged_df.head()

,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species,number
0,1/1/2000,2000,US,FL,28.01863,-82.112864,Monarch Tagged,1
1,1/1/2000,2000,US,FL,28.53834,-81.379237,Monarch Tagged,1
2,1/10/1998,1998,US,KS,37.58613,-99.475674,Monarch Tagged,9
3,1/10/1998,1998,US,TX,25.90175,-97.497484,Monarch Tagged,1
4,1/10/1998,1998,US,TX,29.76043,-95.369803,Monarch Tagged,7


In [23]:
MWatch_Tagged_year_df = MWatch_Tagged_df
# MWatch_Tagged_year_df['years'] = pd.DatetimeIndex(MWatch_Tagged_year_df['eventDate']).year

# Filter Tagging Data By Year - 1996

In [24]:
# filtering by year
MWatch_Tagged_1996_df = MWatch_Tagged_df[MWatch_Tagged_df['year']==1996]
MWatch_Tagged_1996_df.head()

,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species,number
179,10/1/1996,1996,CA,ON,43.45030,-80.483192,Monarch Tagged,1
180,10/1/1996,1996,US,AR,36.36479,-93.567967,Monarch Tagged,13
181,10/1/1996,1996,US,DE,39.78761,-75.696600,Monarch Tagged,2
182,10/1/1996,1996,US,IA,41.82479,-90.647615,Monarch Tagged,13
183,10/1/1996,1996,US,IA,41.82586,-90.758199,Monarch Tagged,1


In [25]:
MWatch_Tagged_1996_df['eventDate']

179      10/1/1996
180      10/1/1996
181      10/1/1996
182      10/1/1996
183      10/1/1996
           ...    
39479     9/9/1996
39480     9/9/1996
39481     9/9/1996
39482     9/9/1996
39483     9/9/1996
Name: eventDate, Length: 3401, dtype: object

In [26]:
MWatch_Tagged_1996_df.shape

(3401, 8)

In [28]:
MWatch_Tagged_1996_df = pd.DataFrame(MWatch_Tagged_1996_df)
MWatch_Tagged_1996_df['month'] = pd.DatetimeIndex(MWatch_Tagged_1996_df['eventDate']).month

In [29]:
MWatch_Tagged_1996_df.head()

,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
179,10/1/1996,1996,CA,ON,43.45030,-80.483192,Monarch Tagged,1,10
180,10/1/1996,1996,US,AR,36.36479,-93.567967,Monarch Tagged,13,10
181,10/1/1996,1996,US,DE,39.78761,-75.696600,Monarch Tagged,2,10
182,10/1/1996,1996,US,IA,41.82479,-90.647615,Monarch Tagged,13,10
183,10/1/1996,1996,US,IA,41.82586,-90.758199,Monarch Tagged,1,10


In [30]:
MWatch_Tagged_1996_df.drop(['year'], axis=1, inplace=True)

In [31]:
MWatch_Tagged_1996_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
179,10/1/1996,CA,ON,43.45030,-80.483192,Monarch Tagged,1,10
180,10/1/1996,US,AR,36.36479,-93.567967,Monarch Tagged,13,10
181,10/1/1996,US,DE,39.78761,-75.696600,Monarch Tagged,2,10
182,10/1/1996,US,IA,41.82479,-90.647615,Monarch Tagged,13,10
183,10/1/1996,US,IA,41.82586,-90.758199,Monarch Tagged,1,10


## January

In [36]:
MWatch_Tagged_1996_month_df =pd.DataFrame(MWatch_Tagged_1996_df)
MWatch_Tagged_1996_1_df = MWatch_Tagged_1996_month_df[MWatch_Tagged_1996_month_df['month']==1]
MWatch_Tagged_1996_1_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month


In [33]:
MWatch_Tagged_1996_1_df.shape

(0, 8)

In [123]:
MWatch_Tagged_1996_1_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number


In [38]:
MWatch_Tagged_1996_1_df = pd.DataFrame(MWatch_Tagged_1996_1_df)
MWatch_Tagged_1996_1_df.drop(['month'], axis=1, inplace=True)

In [39]:
MWatch_Tagged_1996_1_df

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number


In [40]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1996_1_df['decimalLatitude'] = MWatch_Tagged_1996_1_df['decimalLatitude'].astype(float)
MWatch_Tagged_1996_1_df['decimalLongitude'] = MWatch_Tagged_1996_1_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [41]:
geojson = df_to_geojson(MWatch_Tagged_1996_1_df, cols)

In [42]:
file = open("MWatch_Tagged_1996_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [48]:
MWatch_Tagged_1996_month_df =pd.DataFrame(MWatch_Tagged_1996_df)
MWatch_Tagged_1996_2_df = MWatch_Tagged_1996_month_df[MWatch_Tagged_1996_month_df['month']==2]
MWatch_Tagged_1996_2_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month


In [49]:
MWatch_Tagged_1996_2_df.shape

(0, 8)

In [50]:
MWatch_Tagged_1996_2_df = pd.DataFrame(MWatch_Tagged_1996_1_df)
MWatch_Tagged_1996_2_df.drop(['month'], axis=1, inplace=True)

KeyError: "['month'] not found in axis"

In [51]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1996_2_df['decimalLatitude'] = MWatch_Tagged_1996_2_df['decimalLatitude'].astype(float)
MWatch_Tagged_1996_2_df['decimalLongitude'] = MWatch_Tagged_1996_2_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [52]:
geojson = df_to_geojson(MWatch_Tagged_1996_2_df, cols)

In [53]:
file = open("MWatch_Tagged_1996_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [54]:
MWatch_Tagged_1996_month_df =pd.DataFrame(MWatch_Tagged_1996_df)
MWatch_Tagged_1996_3_df = MWatch_Tagged_1996_month_df[MWatch_Tagged_1996_month_df['month']==3]
MWatch_Tagged_1996_3_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month


In [55]:
MWatch_Tagged_1996_3_df.shape

(0, 8)

In [57]:
MWatch_Tagged_1996_3_df = pd.DataFrame(MWatch_Tagged_1996_3_df)
MWatch_Tagged_1996_3_df.drop(['month'], axis=1, inplace=True)

KeyError: "['month'] not found in axis"

In [58]:
MWatch_Tagged_1996_3_df

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number


In [59]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1996_3_df['decimalLatitude'] = MWatch_Tagged_1996_3_df['decimalLatitude'].astype(float)
MWatch_Tagged_1996_3_df['decimalLongitude'] = MWatch_Tagged_1996_3_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [60]:
geojson = df_to_geojson(MWatch_Tagged_1996_3_df, cols)

In [61]:
file = open("MWatch_Tagged_1996_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [62]:
MWatch_Tagged_1996_month_df =pd.DataFrame(MWatch_Tagged_1996_df)
MWatch_Tagged_1996_4_df = MWatch_Tagged_1996_month_df[MWatch_Tagged_1996_month_df['month']==4]
MWatch_Tagged_1996_4_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month


In [ ]:
MWatch_Tagged_1996_4_df.shape

In [63]:
MWatch_Tagged_1996_4_df = pd.DataFrame(MWatch_Tagged_1996_4_df)
MWatch_Tagged_1996_4_df.drop(['month'], axis=1, inplace=True)

In [64]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1996_4_df['decimalLatitude'] = MWatch_Tagged_1996_4_df['decimalLatitude'].astype(float)
MWatch_Tagged_1996_4_df['decimalLongitude'] = MWatch_Tagged_1996_4_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [65]:
geojson = df_to_geojson(MWatch_Tagged_1996_4_df, cols)

In [66]:
file = open("MWatch_Tagged_1996_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [67]:
MWatch_Tagged_1996_month_df =pd.DataFrame(MWatch_Tagged_1996_df)
MWatch_Tagged_1996_5_df = MWatch_Tagged_1996_month_df[MWatch_Tagged_1996_month_df['month']==5]
MWatch_Tagged_1996_5_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month


In [68]:
MWatch_Tagged_1996_5_df.shape

(0, 8)

In [69]:
MWatch_Tagged_1996_5_df =pd.DataFrame(MWatch_Tagged_1996_5_df)
MWatch_Tagged_1996_5_df.drop(['month'], axis=1, inplace=True)

In [70]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1996_5_df['decimalLatitude'] = MWatch_Tagged_1996_5_df['decimalLatitude'].astype(float)
MWatch_Tagged_1996_5_df['decimalLongitude'] = MWatch_Tagged_1996_5_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [71]:
geojson = df_to_geojson(MWatch_Tagged_1996_5_df, cols)

In [72]:
file = open("MWatch_Tagged_1996_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [73]:
MWatch_Tagged_1996_month_df =pd.DataFrame(MWatch_Tagged_1996_df)
MWatch_Tagged_1996_6_df = MWatch_Tagged_1996_month_df[MWatch_Tagged_1996_month_df['month']==6]
MWatch_Tagged_1996_6_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month


In [74]:
MWatch_Tagged_1996_6_df.shape

(0, 8)

In [75]:
MWatch_Tagged_1996_6_df =pd.DataFrame(MWatch_Tagged_1996_6_df)
MWatch_Tagged_1996_6_df.drop(['month'], axis=1, inplace=True)

In [76]:
MWatch_Tagged_1996_6_df

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number


In [77]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1996_6_df['decimalLatitude'] = MWatch_Tagged_1996_6_df['decimalLatitude'].astype(float)
MWatch_Tagged_1996_6_df['decimalLongitude'] = MWatch_Tagged_1996_6_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [78]:
geojson = df_to_geojson(MWatch_Tagged_1996_6_df, cols)

In [79]:
file = open("MWatch_Tagged_1996_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [80]:
MWatch_Tagged_1996_month_df =pd.DataFrame(MWatch_Tagged_1996_df)
MWatch_Tagged_1996_7_df = MWatch_Tagged_1996_month_df[MWatch_Tagged_1996_month_df['month']==7]
MWatch_Tagged_1996_7_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11905,7/14/1996,US,KS,39.20194,-96.304997,Monarch Tagged,4,7
11911,7/19/1996,US,KS,39.20194,-96.304997,Monarch Tagged,4,7


In [81]:
MWatch_Tagged_1996_7_df.shape

(2, 8)

In [82]:
MWatch_Tagged_1996_7_df

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11905,7/14/1996,US,KS,39.20194,-96.304997,Monarch Tagged,4,7
11911,7/19/1996,US,KS,39.20194,-96.304997,Monarch Tagged,4,7


In [83]:
MWatch_Tagged_1996_7_df =pd.DataFrame(MWatch_Tagged_1996_7_df)
MWatch_Tagged_1996_7_df.drop(['month'], axis=1, inplace=True)

In [84]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1996_7_df['decimalLatitude'] = MWatch_Tagged_1996_7_df['decimalLatitude'].astype(float)
MWatch_Tagged_1996_7_df['decimalLongitude'] = MWatch_Tagged_1996_7_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [85]:
MWatch_Tagged_1996_7_df

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number
11905,7/14/1996,US,KS,39.20194,-96.304997,Monarch Tagged,4
11911,7/19/1996,US,KS,39.20194,-96.304997,Monarch Tagged,4


In [88]:
geojson = df_to_geojson(MWatch_Tagged_1996_7_df, cols)

In [89]:
file = open("MWatch_Tagged_1996_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [91]:
MWatch_Tagged_1996_month_df =pd.DataFrame(MWatch_Tagged_1996_df)
MWatch_Tagged_1996_8_df = MWatch_Tagged_1996_month_df[MWatch_Tagged_1996_month_df['month']==8]
MWatch_Tagged_1996_8_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
16103,8/8/1996,US,MN,45.119410,-93.402453,Monarch Tagged,1,8
16104,8/8/1996,US,TX,29.370886,-100.895867,Monarch Tagged,23,8
16123,8/9/1996,US,MN,45.072460,-93.455788,Monarch Tagged,1,8
16124,8/9/1996,US,MN,45.119410,-93.402453,Monarch Tagged,1,8
16125,8/9/1996,US,TX,29.370886,-100.895867,Monarch Tagged,15,8


In [92]:
MWatch_Tagged_1996_8_df.shape

(246, 8)

In [93]:
MWatch_Tagged_1996_8_df = pd.DataFrame(MWatch_Tagged_1996_8_df)
MWatch_Tagged_1996_8_df.drop(['month'], axis=1, inplace=True)

In [94]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1996_8_df['decimalLatitude'] = MWatch_Tagged_1996_8_df['decimalLatitude'].astype(float)
MWatch_Tagged_1996_8_df['decimalLongitude'] = MWatch_Tagged_1996_8_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [95]:
geojson = df_to_geojson(MWatch_Tagged_1996_8_df, cols)

In [96]:
file = open("MWatch_Tagged_1996_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [97]:
MWatch_Tagged_1996_month_df =pd.DataFrame(MWatch_Tagged_1996_df)
MWatch_Tagged_1996_9_df = MWatch_Tagged_1996_month_df[MWatch_Tagged_1996_month_df['month']==9]
MWatch_Tagged_1996_9_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
39479,9/9/1996,US,WI,44.740810,-89.063165,Monarch Tagged,6,9
39480,9/9/1996,US,WV,37.801788,-80.445630,Monarch Tagged,4,9
39481,9/9/1996,US,WV,38.642890,-79.331153,Monarch Tagged,8,9
39482,9/9/1996,US,WV,39.286480,-80.256198,Monarch Tagged,19,9
39483,9/9/1996,US,WV,39.440930,-78.973916,Monarch Tagged,1,9


In [98]:
MWatch_Tagged_1996_9_df.shape

(2001, 8)

In [99]:
MWatch_Tagged_1996_9_df = pd.DataFrame(MWatch_Tagged_1996_9_df)
MWatch_Tagged_1996_9_df.drop(['month'], axis=1, inplace=True)

In [100]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1996_9_df['decimalLatitude'] = MWatch_Tagged_1996_9_df['decimalLatitude'].astype(float)
MWatch_Tagged_1996_9_df['decimalLongitude'] = MWatch_Tagged_1996_9_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [101]:
geojson = df_to_geojson(MWatch_Tagged_1996_9_df, cols)

In [102]:
file = open("MWatch_Tagged_1996_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [103]:
MWatch_Tagged_1996_month_df =pd.DataFrame(MWatch_Tagged_1996_df)
MWatch_Tagged_1996_10_df = MWatch_Tagged_1996_month_df[MWatch_Tagged_1996_month_df['month']==10]
MWatch_Tagged_1996_10_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
10098,10/9/1996,US,TX,32.93429,-97.078065,Monarch Tagged,17,10
10099,10/9/1996,US,TX,32.96179,-96.829168,Monarch Tagged,52,10
10100,10/9/1996,US,TX,32.97564,-96.889964,Monarch Tagged,10,10
10101,10/9/1996,US,WI,44.81135,-91.498494,Monarch Tagged,2,10
10102,10/9/1996,US,WV,39.35825,-81.167847,Monarch Tagged,2,10


In [104]:
MWatch_Tagged_1996_10_df.shape

(1026, 8)

In [105]:
MWatch_Tagged_1996_10_df = pd.DataFrame(MWatch_Tagged_1996_10_df)
MWatch_Tagged_1996_10_df.drop(['month'], axis=1, inplace=True)

In [106]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1996_10_df['decimalLatitude'] = MWatch_Tagged_1996_10_df['decimalLatitude'].astype(float)
MWatch_Tagged_1996_10_df['decimalLongitude'] = MWatch_Tagged_1996_10_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [107]:
geojson = df_to_geojson(MWatch_Tagged_1996_10_df, cols)

In [108]:
file = open("MWatch_Tagged_1996_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [109]:
MWatch_Tagged_1996_month_df =pd.DataFrame(MWatch_Tagged_1996_df)
MWatch_Tagged_1996_11_df = MWatch_Tagged_1996_month_df[MWatch_Tagged_1996_month_df['month']==11]
MWatch_Tagged_1996_11_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11281,11/9/1996,US,IA,41.85718,-90.925083,Monarch Tagged,19,11
11282,11/9/1996,US,SC,32.71183,-79.952113,Monarch Tagged,1,11
11283,11/9/1996,US,TX,29.34746,-99.141425,Monarch Tagged,2,11
11284,11/9/1996,US,TX,30.19021,-98.086678,Monarch Tagged,1,11
11285,11/9/1996,US,TX,30.26715,-97.743061,Monarch Tagged,1,11


In [110]:
MWatch_Tagged_1996_11_df.shape

(107, 8)

In [111]:
MWatch_Tagged_1996_11_df = pd.DataFrame(MWatch_Tagged_1996_11_df)
MWatch_Tagged_1996_11_df.drop(['month'], axis=1, inplace=True)

In [112]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1996_11_df['decimalLatitude'] = MWatch_Tagged_1996_11_df['decimalLatitude'].astype(float)
MWatch_Tagged_1996_11_df['decimalLongitude'] = MWatch_Tagged_1996_11_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [113]:
geojson = df_to_geojson(MWatch_Tagged_1996_11_df, cols)

In [114]:
file = open("MWatch_Tagged_1996_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [116]:
MWatch_Tagged_1996_month_df =pd.DataFrame(MWatch_Tagged_1996_df)
MWatch_Tagged_1996_12_df = MWatch_Tagged_1996_month_df[MWatch_Tagged_1996_month_df['month']==12]
MWatch_Tagged_1996_12_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11499,12/7/1996,US,SC,32.71183,-79.952113,Monarch Tagged,2,12
11519,12/9/1996,US,IA,41.85718,-90.925083,Monarch Tagged,79,12
11520,12/9/1996,US,KS,39.20194,-96.304997,Monarch Tagged,16,12
11521,12/9/1996,US,MD,39.10344,-77.290817,Monarch Tagged,3,12
11522,12/9/1996,US,MN,45.11941,-93.402453,Monarch Tagged,2,12


In [117]:
MWatch_Tagged_1996_12_df.shape

(19, 8)

In [118]:
MWatch_Tagged_1996_12_df = pd.DataFrame(MWatch_Tagged_1996_12_df)
MWatch_Tagged_1996_12_df.drop(['month'], axis=1, inplace=True)

In [120]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1996_12_df['decimalLatitude'] = MWatch_Tagged_1996_12_df['decimalLatitude'].astype(float)
MWatch_Tagged_1996_12_df['decimalLongitude'] = MWatch_Tagged_1996_12_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [121]:
geojson = df_to_geojson(MWatch_Tagged_1996_12_df, cols)

In [122]:
file = open("MWatch_Tagged_1996_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()